# Notebook de mejora

Este notebook es un ejemplo cómo se puede mejorar los hiperparámetros de un algoritmo, y medir la importancia de atributos https://www.kaggle.com/c/house-prices-advanced-regression-techniques/ de la práctica 3 de Inteligencia de Negocio. 

Incluyo el código del notebook anterior, no se explicarán.

Es un ejemplo que es claramente mejorable, se deja a el/la estudiante el mejorarlo para obtener mejores resultados.

In [53]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, RobustScaler


Cambiamos preprocesamiento, lo hacemos con los dos conjuntos juntos.

In [70]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", na_values="NaN") # Definimos na_values para identificar bien los valores perdidos

train['is_train'] = 1

if 'Id' in train:
    train.drop('Id', axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

test = pd.read_csv("test.csv", na_values="NaN")
test_ids = test.Id
test = test.drop('Id', axis=1)

test['is_train'] = 0


input_all = pd.concat([train, test])

col_cat = list(input_all.select_dtypes(exclude=np.number).columns)

from sklearn.impute import SimpleImputer

# Valores categóricos por el más frecuente
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit_transform(input_all[col_cat])

input_all = pd.get_dummies(input_all, columns=col_cat, drop_first=True)
input_all['SalePrice'] = np.log1p(input_all['SalePrice'])



# Valores numéricos por KNN
col_num = list(train.select_dtypes(include=np.number).columns)
col_num.remove('SalePrice')

# Configuración del imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos según tus necesidades

# Imputación para el conjunto de entrenamiento
input_all[col_num] = knn_imputer.fit_transform(input_all[col_num])


# Split the data back into train and test sets
train = input_all[input_all['is_train'] == 1].drop(['is_train'], axis=1)
test = input_all[input_all['is_train'] == 0].drop(['is_train'], axis=1)

from sklearn.preprocessing import LabelEncoder
labelers = {}
test_l = test.copy()
train_l = train.copy()


y_train = train_l.SalePrice
X_train = train_l.drop('SalePrice', axis=1)

if 'Id' in test_l:
    test_l.drop('Id', axis=1, inplace=True)

X_test = test_l

In [71]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1456,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,163.0,...,0,0,0,0,1,0,0,0,1,0
1457,70.0,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1458,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,1029.0,...,0,0,0,0,1,0,0,0,1,0


In [72]:
y_train

0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1460, dtype: float64

In [73]:
# Define the cross-validation strategy
cv = KFold(n_splits=10, shuffle=True, random_state=124345)

In [74]:
from sklearn.compose import make_column_selector

cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)

In [75]:
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_linear_processor = OneHotEncoder(handle_unknown="ignore")
num_linear_processor = StandardScaler()

linear_preprocessor = make_column_transformer(
    (num_linear_processor, num_selector), (cat_linear_processor, cat_selector)
)
linear_preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000026467C43310>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000026467C43160>)])

In [76]:
cat_tree_processor = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1,
    encoded_missing_value=-2,
)

num_tree_processor = SimpleImputer(strategy="mean", add_indicator=True)

tree_preprocessor = make_column_transformer(
    (num_tree_processor, num_selector), (cat_tree_processor, cat_selector)
)
tree_preprocessor

ColumnTransformer(transformers=[('simpleimputer',
                                 SimpleImputer(add_indicator=True),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000026467C43310>),
                                ('ordinalencoder',
                                 OrdinalEncoder(encoded_missing_value=-2,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000026467C43160>)])

In [77]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = make_pipeline(RobustScaler(), RandomForestRegressor(criterion='squared_error', max_depth=20, random_state=42))
rf_pipeline

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, random_state=42))])

In [78]:
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# KNN pipeline
knn_pipeline = make_pipeline(RobustScaler(), KNeighborsRegressor())

knn_pipeline

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('kneighborsregressor', KNeighborsRegressor())])

In [79]:
# GradientBoostingRegressor pipeline
gradient_boosting_pipeline = make_pipeline(
    RobustScaler(), GradientBoostingRegressor(random_state=42, learning_rate=0.15, max_depth=4, n_estimators=333)
)

gradient_boosting_pipeline

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(learning_rate=0.15, max_depth=4,
                                           n_estimators=333,
                                           random_state=42))])

In [80]:
# XGBoost pipeline
xgboost_pipeline = make_pipeline(
    RobustScaler(), XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, gamma=0.05, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.5, n_estimators=3000,
                             reg_alpha=0.5, reg_lambda=0.8,
                             subsample=0.5,
                             random_state =42, seed=42)
)

xgboost_pipeline

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.5, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=0.05, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=1.5, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=3000, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [81]:
from sklearn.linear_model import LassoCV

lasso_pipeline = make_pipeline(RobustScaler(), LassoCV(max_iter=3000))
lasso_pipeline

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('lassocv', LassoCV(max_iter=3000))])

In [82]:
import lightgbm as lgb

# LightGBM pipeline
lightgbm_pipeline = make_pipeline(
    RobustScaler(), lgb.LGBMRegressor(objective='regression', verbose=-1, bagging_fraction = 0.8, bagging_seed=9, seed=42, n_estimators=300, max_depth=4, learning_rate=0.15)
)

lightgbm_pipeline

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.8, bagging_seed=9,
                               learning_rate=0.15, max_depth=4,
                               n_estimators=300, objective='regression',
                               seed=42, verbose=-1))])

In [83]:
# Define the base models
base_models = [
    ('random_forest', rf_pipeline),
    ('knn', knn_pipeline),
    ('gradient_boosting', gradient_boosting_pipeline),
    ('xgboost', xgboost_pipeline),
    ('lasso', lasso_pipeline),
    ('lightgbm', lightgbm_pipeline)
]

stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=Ridge())
stacking_regressor

StackingRegressor(estimators=[('random_forest',
                               Pipeline(steps=[('robustscaler', RobustScaler()),
                                               ('randomforestregressor',
                                                RandomForestRegressor(max_depth=20,
                                                                      random_state=42))])),
                              ('knn',
                               Pipeline(steps=[('robustscaler', RobustScaler()),
                                               ('kneighborsregressor',
                                                KNeighborsRegressor())])),
                              ('gradient_boosting',
                               Pipeline(steps=[('robustscaler', RobustScaler()),
                                               (...
                                                             random_state=42, ...))])),
                              ('lasso',
                               Pipeline(steps=[('robustscaler', RobustScaler()),
                                               ('lassocv',
                                                LassoCV(max_iter=3000))])),
                              ('lightgbm',
                               Pipeline(steps=[('robustscaler', RobustScaler()),
                                               ('lgbmregressor',
                                                LGBMRegressor(bagging_fraction=0.8,
                                                              bagging_seed=9,
                                                              learning_rate=0.15,
                                                              max_depth=4,
                                                              n_estimators=300,
                                                              objective='regression',
                                                              seed=42,
                                                              verbose=-1))]))],
                  final_estimator=Ridge())

In [84]:
# Validate each base model using RMSE through cross-validation
for name, model in base_models:
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse_scores = np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=kfold))
    print(f'RMSE for {name}: {rmse_scores.mean()}')
    


RMSE for random_forest: 0.14145910390059407
RMSE for knn: 0.24439719247354189
RMSE for gradient_boosting: 0.1292578050351606
RMSE for xgboost: 0.12525479370041667
RMSE for lasso: 0.15048647763217507
RMSE for lightgbm: 0.13029689244072495


In [85]:
from sklearn.metrics import mean_squared_error

# Validate the stacking regressor using RMSLE through cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
rmsle_scores = np.sqrt(-cross_val_score(stacking_regressor, X_train, y_train, scoring="neg_mean_squared_error", cv=kfold))
print(f'RMSLE for Stacking Regressor: {rmsle_scores.mean()}')

# Train the stacking regressor on the entire training data
stacking_regressor.fit(X_train, y_train)

# Display RMSLE for the stacking model on the training data
y_pred_stacking = stacking_regressor.predict(X_train)
rmsle_stacking = np.sqrt(mean_squared_error(y_train, y_pred_stacking))
print(f'RMSLE for Stacking Regressor on training data: {rmsle_stacking}')

RMSLE for Stacking Regressor: 0.12458891386481585
RMSLE for Stacking Regressor on training data: 0.05829616328390982


In [92]:
X_test = X_test.drop('SalePrice', axis=1)

In [93]:
pred = stacking_regressor.predict(X_test)

In [96]:
salida = pd.DataFrame({'Id': test_ids, 'SalePrice': np.expm1(pred)})
salida.to_csv("Prueba11.csv", index=False)